In [4]:
from IPython.display import display, Markdown

import pandas as pd
import gc
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score

from sklearn.metrics import roc_auc_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

from sklearn.tree import plot_tree

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler, OneHotEncoder

# XGBoost
import xgboost as xgb

import numpy as np

import matplotlib.pyplot as plt

import itertools


## ----------------------------------------------------------------
# Intento 1

LOAD DATA y drop atribute

In [6]:
def load_comp():
    # Load the competition data
    comp_data = pd.read_csv("../competition_data.csv")
    comp_data = comp_data.drop(
        columns=[
            "benefit", 
            "etl_version", 
            "uid",
            "date",
            "deal_print_id",
            "full_name",
            "main_picture",
            "warranty",
            "tags",
            "print_server_timestamp",
            "print_position",
            "title",
            "accepts_mercadopago"
        ]
    )
    # OHE comp_data columns
    comp_data = pd.get_dummies(comp_data, 
        sparse=True,
        columns=[
            #"category_id", 
            #"domain_id", 
            "logistic_type", 
            "platform", 
            "site_id"
        ],
        dtype=int
    )
    #comp_data["accepts_mercadopago"] = comp_data["accepts_mercadopago"].astype(int)
    # todos aceptan mercadopago
    comp_data["boosted"] = comp_data["boosted"].astype(int)
    comp_data["free_shipping"] = comp_data["free_shipping"].astype(int)
    comp_data["fulfillment"] = comp_data["fulfillment"].astype(int)

    comp_data["is_pdp"].fillna(0, inplace=True)
    comp_data["is_pdp"] = comp_data["is_pdp"].astype(int)
    #comp_data["warranty"] = comp_data["warranty"].astype(int)

    # comp_data["listing_type_id"] to 0 if gold_special, 1 if gold_pro.
    comp_data["listing_type_id"] = comp_data["listing_type_id"].apply(lambda x: 0 if x == "gold_special" else 1)
    
    # Label encode category_id and domain_id
    # comp_data["category_id"] = comp_data["category_id"].astype("category")#.cat.codes
    #comp_data["domain_id"] = comp_data["domain_id"].astype("category")#.cat.codes

    # sklearn LabelEncoder for category_id and domain_id
    
    comp_data["category_id"] = LabelEncoder().fit_transform(comp_data["category_id"]).astype(int)
    comp_data["domain_id"] = LabelEncoder().fit_transform(comp_data["domain_id"]).astype(int)
    comp_data["item_id"] = LabelEncoder().fit_transform(comp_data["item_id"]).astype(int)

    comp_data["price_diff"] = comp_data["price"] - comp_data["original_price"]
    comp_data["cheaper_than_original"] = comp_data["price_diff"].apply(lambda x: 1 if x < 0 else 0)
    comp_data["price_diff"] = comp_data["price_diff"].apply(lambda x: abs(x)).astype(int)

    #comp_data["cheaper_than_avg"] = comp_data["price"] - comp_data["avg_asp_item_domain"]
    #comp_data["cheaper_than_avg"] = comp_data["cheaper_than_avg"].apply(lambda x: 1 if x < 0 else 0).astype(int)

    # Drop useless columns
    # comp_data = comp_data.drop(
    #     columns=[
    #     ]
    # )

    return comp_data
comp_data = load_comp()
comp_data["item_id"].value_counts()

item_id
14540    717
11932    701
27372    622
14900    535
12183    517
        ... 
23949      1
3863       1
14492      1
18426      1
11527      1
Name: count, Length: 27695, dtype: int64

SPLIT DATA

In [7]:
# Split into training and evaluation samples
comp_data = load_comp()

display(
    Markdown(
        "\n".join(
            [
                "- **{}** ({})".format(col, dtype) for col, dtype in
                zip(comp_data.columns, comp_data.dtypes)
            ]
        )
    )
)

full_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]

# Get number column names for full_data
#print(full_data.columns)
#num_cols = full_data.select_dtypes(include='number').columns
#print(num_cols)
# difference between full_data_cols and num_cols
# print(set(full_data.columns) - set(num_cols))

del comp_data
gc.collect()

train_data, test_data = train_test_split(full_data, test_size=0.20, train_size=0.80, random_state=42)

y_train = train_data["conversion"]
X_train = train_data.drop(columns=["conversion", "ROW_ID"])
X_train = X_train.select_dtypes(include='number')

y_test = test_data["conversion"]
X_test = test_data.drop(columns=["conversion", "ROW_ID"])
X_test = X_test.select_dtypes(include='number')

del train_data
del test_data

gc.collect()

- **available_quantity** (int64)
- **avg_gmv_item_domain_30days** (float64)
- **avg_gmv_item_sel** (float64)
- **avg_gmv_seller_bday** (float64)
- **avg_qty_orders_item_domain_30days** (float64)
- **avg_qty_orders_item_sel_30days** (float64)
- **avg_si_item_sel_30day** (float64)
- **boosted** (int32)
- **category_id** (int32)
- **conversion** (float64)
- **domain_id** (int32)
- **free_shipping** (int32)
- **fulfillment** (int32)
- **health** (float64)
- **is_pdp** (int32)
- **product_id** (float64)
- **item_id** (int32)
- **listing_type_id** (int64)
- **offset** (int64)
- **original_price** (int64)
- **price** (int64)
- **qty_items_dom** (float64)
- **qty_items_sel** (float64)
- **sold_quantity** (int64)
- **total_asp_item_domain_30days** (float64)
- **total_asp_item_sel_30days** (float64)
- **total_gmv_domain_bday** (float64)
- **total_gmv_item_30days** (float64)
- **total_items_domain** (int64)
- **total_items_seller** (int64)
- **total_orders_domain_30days** (float64)
- **total_orders_item_30days** (float64)
- **total_orders_sel_30days** (float64)
- **total_si_domain_30days** (float64)
- **total_si_item_30days** (float64)
- **total_si_sel_30days** (float64)
- **total_visits_domain** (int64)
- **total_visits_item** (int64)
- **total_visits_seller** (int64)
- **user_id** (float64)
- **ROW_ID** (float64)
- **logistic_type_cross_docking** (Sparse[int32, 0])
- **logistic_type_custom** (Sparse[int32, 0])
- **logistic_type_default** (Sparse[int32, 0])
- **logistic_type_drop_off** (Sparse[int32, 0])
- **logistic_type_fulfillment** (Sparse[int32, 0])
- **logistic_type_not_specified** (Sparse[int32, 0])
- **logistic_type_xd_drop_off** (Sparse[int32, 0])
- **platform_/mobile/android** (Sparse[int32, 0])
- **platform_/mobile/ios** (Sparse[int32, 0])
- **platform_/web/desktop** (Sparse[int32, 0])
- **platform_/web/mobile** (Sparse[int32, 0])
- **site_id_MLA** (Sparse[int32, 0])
- **price_diff** (int32)
- **cheaper_than_original** (int64)

0

In [8]:
atributes_to_poly = [
                        "available_quantity", 
                        "avg_gmv_item_domain_30days", 
                        "avg_gmv_item_sel", 
                        "avg_gmv_seller_bday", 
                        "avg_qty_orders_item_domain_30days", 
                        "avg_qty_orders_item_sel_30days", 
                        "avg_si_item_sel_30day",
                        "original_price",
                        "price",
                    ]

imputed_poly_features = make_pipeline(
  #SimpleImputer(),
  #PolynomialFeatures(),
  StandardScaler(),
  xgb.XGBClassifier(
        missing=np.nan,
        n_jobs=-1,
        tree_method='gpu_hist',
    )
)

# Custom Layer Starts Here:
pl = PolynomialFeatures(2, interaction_only=True, include_bias=False)

#Input X contains NaN.

imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_test.copy()
print(imputed_X_train_plus.shape[0] == X_train.shape[0])

# Make new columns indicating what will be imputed
cols_with_missing = (col for col in X_train.columns
                        if X_train[col].isnull().any())

for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()

poly_X_train = imputed_X_train_plus[atributes_to_poly]
poly_X_test = imputed_X_test_plus[atributes_to_poly]
print(poly_X_train.shape[0] == X_train.shape[0])

poly_X_train = my_imputer.fit_transform(poly_X_train)
poly_X_test = my_imputer.transform(poly_X_test)
print(poly_X_train.shape[0] == X_train.shape[0])

poly_X_train = pl.fit_transform(poly_X_train)
poly_X_test = pl.fit_transform(poly_X_test)
print(poly_X_train.shape[0] == X_train.shape[0])

new_X_train = np.concatenate((X_train.drop(atributes_to_poly, axis=1), poly_X_train), axis=1)
new_X_test = np.concatenate((X_test.drop(atributes_to_poly, axis=1), poly_X_test), axis=1)
print(new_X_train.shape[0] == X_train.shape[0])
# Custom Layer Ends Here


imputed_poly_features.fit(new_X_train, y_train)

roc_auc_score(y_test, imputed_poly_features.predict_proba(new_X_test)[:, imputed_poly_features.classes_ == 1])
# Decision Tree Classifier with K-Fold Cross Validation and Randomized Search CV

dtc = make_pipeline(
    SimpleImputer(),
    RandomizedSearchCV(
        DecisionTreeClassifier(random_state=2345),
        param_distributions={
            "max_depth": [2, 4, 8, 16, 32, 64, 128, 256, 512, None],
            "min_samples_split": [2, 4, 8, 16, 32, 64, 128, 256, 512],
            "min_samples_leaf": [2, 4, 8, 16, 32, 64, 128, 256, 512],
            "max_features": [2, 4, 8, 16, 32, 64, 128, 256, 512, None],
        },
        n_iter=100,
        cv=3,
        random_state=2345,
        n_jobs=-1,
        verbose=0,
    )
)

kf = KFold(n_splits=5, random_state=2345, shuffle=True)
roc_aucs = []

for train_index, valid_index in kf.split(X_train):
    X_train_kf, X_valid_kf = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_kf, y_valid_kf = y_train.iloc[train_index], y_train.iloc[valid_index]

    dtc.fit(X_train_kf, y_train_kf)
    # Metric: roc_auc_score
    roc_aucs.append(roc_auc_score(y_valid_kf, dtc.predict_proba(X_valid_kf)[:, dtc.classes_ == 1]))
    print(f"ROC-AUC split {len(roc_aucs)}: ", roc_aucs[-1])

print("ROC-AUC mean: ", sum(roc_aucs) / len(roc_aucs))    

dtc.fit(X_train, y_train)

print("ROC-AUC: ", roc_auc_score(y_test, dtc.predict_proba(X_test)[:, dtc.classes_ == 1]))

True
True
True
True
True


c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ROC-AUC split 1:  0.7116231505769222


c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ROC-AUC split 2:  0.7235289611958904


...

c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ROC-AUC split 4:  0.6869342415878192


c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ROC-AUC split 5:  0.7223620781213635
ROC-AUC mean:  0.7136215825039838
ROC-AUC:  0.7246614377882451


c:\Users\oltie\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [9]:
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_preds})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("exp_oli_1.csv", sep=",", index=False)

## ----------------------------------------------------------------
# Intento 2

### KFOLD CLASE 3

In [ ]:
# k-fold cross validation
kcv = KFold(n_splits=10, shuffle=True)
rmse_kcv1 = []
for train_ix, test_ix in tqdm(kcv.split(X_train), total=10):
    X_train_red, X_val = X_train.iloc[train_ix,:], X_train.iloc[test_ix,:]
    y_train_red, y_val = y_train[train_ix], y_train[test_ix]
    tree = DecisionTreeRegressor(max_depth=10, random_state=TREE_SEED)
    tree.fit(X_train_red, y_train_red)
    preds_val = tree.predict(X_val)
    rmse_kcv1.append(math.sqrt(mean_squared_error(y_val, preds_val)))

print(f"RMSE estimado mediante k-fold CV (Versión 1): {mean(rmse_kcv1):.2f}")

# Validación cruzada k-fold utilizando cross_val_score
kcv = KFold(n_splits=10, shuffle=True)
tree = DecisionTreeRegressor(max_depth=10, random_state=TREE_SEED)
rmse_kcv2 = cross_val_score(tree, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kcv, n_jobs=-1)

print(f"RMSE estimado mediante k-fold CV (Versión 2): {-1 * mean(rmse_kcv2):.2f}")


### Model selection
no se si usar


In [ ]:
# Training, holdout y testing sets
size_val = math.ceil(0.1 * X_train.shape[0])
size_test = math.ceil(0.1 * X_train.shape[0])

X_train_red, X_test, y_train_red, y_test = train_test_split(X_train, y_train, test_size=size_test)
X_train_red, X_val, y_train_red, y_val = train_test_split(X_train_red, y_train_red, test_size=size_val)

# Prueba con distintos niveles de profundidad (model selection)
exp_results = []
for md in tqdm(range(1, 51)):
    tree = DecisionTreeRegressor(max_depth=md, random_state=TREE_SEED)
    tree.fit(X_train_red, y_train_red)
    preds_val = tree.predict(X_val)
    exp_results.append({"max_depth": md,
                        "rmse_val": math.sqrt(mean_squared_error(y_val, preds_val))})

exp_results = pd.DataFrame(exp_results)
plot_exp(exp_results)

# Se entrena el árbol con la mejor profundidad encontrada sobre train set + validation set
best_md = exp_results[exp_results["rmse_val"].min() == exp_results["rmse_val"]]
best_md = best_md.sort_values("max_depth").iloc[0,:]
print(f"Performance del mejor modelos: {best_md['rmse_val']:.2f}")

tree = DecisionTreeRegressor(max_depth=int(best_md["max_depth"]), random_state=TREE_SEED)
tree.fit(pd.concat([X_train_red, X_val], axis=0),
         pd.concat([y_train_red, y_val], axis=0))
preds_test = tree.predict(X_test)

print(f"RMSE estimado en test: {math.sqrt(mean_squared_error(y_test, preds_test)):.2f}")

# Evaluación final en el conjunto de evaluación (model assestment)
tree = DecisionTreeRegressor(max_depth=int(best_md["max_depth"]), random_state=TREE_SEED)
tree.fit(X_train, y_train)
preds_eval = tree.predict(X_eval)
_, _, _, y_eval = load_data()

print(f"Performance de evaluación: {math.sqrt(mean_squared_error(y_eval, preds_eval)):.2f}")

### OHE

In [ ]:
# One-hot-encoding
# Opción 1: utilizar get_dummies de pandas
X_train_ohe_v1 = pd.get_dummies(X_train).head()  # Y ahora cómo lo aplico en validación? (debería unir antes los datos)

# Opción 2: utilizar onehotencoder de sklearn (más conveniente para producción)

# Sobre training
X_train_obj = X_train.select_dtypes(include=["object"])
X_train_num = X_train.select_dtypes(include=["int", "float"])

scaler = StandardScaler()
encoder = OneHotEncoder(sparse_output=True)
X_train_ohe_obj = encoder.fit_transform(X_train_obj)
X_train_ohe_v2 = hstack([X_train_ohe_obj, scaler.fit_transform(X_train_num)])

# Sobre validation
X_val_obj = X_val.select_dtypes(include=["object"])
X_val_num = X_val.select_dtypes(include=["int", "float"])

X_val_ohe_obj = encoder.transform(X_val_obj)
X_val_ohe_v2 = hstack([X_val_ohe_obj, scaler.transform(X_val_num)])

# Entrenamos un modelo de regresión logística sobre los datos con OHE sin escalado
lr_exp = []
for C in tqdm([0.001, 0.0025, 0.01, 0.05, 0.075, 0.1, 0.25]):
    lr = LogisticRegression(C=C, max_iter=2000)
    lr.fit(X_train_ohe_v2, y_train)
    preds = lr.predict_proba(X_val_ohe_v2)
    preds = preds[:, lr.classes_ == "yes"]
    auc = roc_auc_score(y_val == "yes", preds)
    lr_exp.append({"C": C, "auc_val": auc})

lr_exp = pd.DataFrame(lr_exp)
print(lr_exp)